In [2]:
%matplotlib inline
from skimage import * # utils like img_as_ubyte
from skimage import io
from skimage.io import imread, imsave, imshow
from skimage.transform import resize

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure, subplot, plot, yticks, xticks
from matplotlib.pyplot import legend, colorbar, xlim, ylim
# projection='3d'
from mpl_toolkits.mplot3d.axes3d import Axes3D

from leicaexperiment import Experiment

from math import ceil
import numpy as np

In [ ]:
def imshow_range(img):
    "Show only range of intensities in image."
    imgplot = io.imshow(img, vmin=img.min(), vmax=img.max())
    colorbar()
    return imgplot

In [3]:
def imshow_downscale(img):
    "matplotlib is slooooow, showing downscaled images is speedier"
    if type(img) == str:
        img = imread(img)
    y,x = img.shape
    # downscale factor
    k = ceil(y//512+0.1)
    imshow(resize(img, (y//k, x//k)))

    # real ticks
    if y > x:
        # less ticks if not square image
        y_step = y//6
        x_step = int(y_step*x/y)
    if x < y:
        y_step = y//6
        x_step = int(y_step*y/x)
    else:
        y_step = y//6
        x_step = x//6

    yticks(range(0,y//k,y_step//k), [str(y) for y in range(0,y,y_step)])
    xticks(range(0,x//k,x_step//k), [str(x) for x in range(0,x,x_step)]);

In [4]:
def side_by_side(*images):
    "view images side by side"
    size = len(images)
    figure(figsize=(size*5,5))
    for i,img in enumerate(images):
        subplot(1, size, i+1)
        imshow_downscale(img)

In [5]:
def merge(img1, img2, dy, dx):
    y,x = img1.shape
    yy,xx = img2.shape
    
    if dy >= 0:
        y_pos1 = slice(0, y)
        y_pos2 = slice(dy, yy+dy)
    if dy < 0:
        y_pos1 = slice(-dy, y-dy)
        y_pos2 = slice(0, yy)
    if dx >= 0:
        x_pos1 = slice(0, x)
        x_pos2 = slice(dx, xx+dx)
    if dx < 0:
        x_pos1 = slice(-dx, x-dx)
        x_pos2 = slice(0, xx)
    
    Y = max((y_pos1.stop, y_pos2.stop)) - min((y_pos1.start, y_pos2.start))
    X = max((x_pos1.stop, x_pos2.stop)) - min((x_pos1.start, x_pos2.start))
    
    merged = np.zeros((Y,X), dtype=np.int)
    merged[y_pos1,x_pos1] += img1
    merged[y_pos2,x_pos2] += img2
    
    # average the overlap
    overlap = (slice(dy, y), slice(dx, x))
    merged[overlap] //= 2
    
    return merged.astype(np.uint8)

In [6]:
def image_axis(ax, ticks=None):
    "Sets tick outward and x ticks on the top"
    ax.xaxis.set_label_position('top') 
    ax.xaxis.tick_top()
    ax.xaxis.set_tick_params(direction='out')

    ax.yaxis.tick_left()
    ax.yaxis.set_tick_params(direction='out')
    
    if ticks:
        ax.yaxis.set_ticks(ticks)
        ax.xaxis.set_ticks(ticks)

In [27]:
MARGIN = 5 # mm
B5 = (250, 176) # y,x mm
B5_HEIGTH = (B5[0] - 2*MARGIN) / 25.4 # inches
B5_WIDTH = (B5[1] - 2*MARGIN) / 25.4 # inches

def savefig(filename, **kwargs):
    "short hand to prevent 600dpi in notebooks"
    fig = plt.gcf()
    x,y = fig.get_size_inches()
    scale = B5_WIDTH / x
    size = B5_WIDTH, y*scale
    fig.set_size_inches(size)
    plt.savefig(filename, dpi=600, bbox_inches='tight', **kwargs)

    fig.set_size_inches((x,y))

In [ ]:
# ranges for images
def range_img(n):
    "for use with mpl ticks. n = img size"
    return range(0, n+1, (n+1)//4)

In [1]:
def show_all_images(images):
    from skimage import viewer
    v = viewer.viewers.CollectionViewer(images)
    v.show()

/Users/arve/.virtualenvs/3.4/lib/python3.4/site-packages/skimage/viewer/utils/core.py:11: UserWarning: Recommended matplotlib backend is `Agg` for full skimage.viewer functionality.
  warnings.warn("Recommended matplotlib backend is `Agg` for full "
